In [15]:
import os
import dash
from dash import  dcc,html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np


In [16]:
# Initialize Dash app
app = dash.Dash()
server = app.server

# Load stock dataset
df_nse = pd.read_csv("./nse.csv")
df_nse["Date"] = pd.to_datetime(df_nse["Date"], format="%Y-%m-%d")
df_nse.set_index("Date", inplace=True)

# Prepare data
new_data = pd.DataFrame(df_nse["Close"]).reset_index()
dataset = new_data["Close"].values.reshape(-1, 1)

# Split data
train_size = 987
train, valid = dataset[:train_size], dataset[train_size:]

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Load or train model
if os.path.exists("model.keras"):
    print("Loading existing model...")
    model = load_model("model.keras")
else:
    print(" Model file not found! Please train the model first.")

Loading existing model...


In [17]:
# Prepare input for prediction using only the "Close" data
inputs = dataset[len(dataset) - len(valid) - 60:]
inputs = scaler.transform(inputs)

X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Predict closing prices
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

# Convert predictions to DataFrame
valid = new_data.iloc[train_size:].copy()
valid.loc[:, 'Predictions'] = closing_price

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step


In [18]:
# Load stock data for Facebook (ensure the file exists)
if os.path.exists("./stock_data.csv"):
    df = pd.read_csv("./stock_data.csv")
else:
    print("⚠️ stock_data.csv not found!")


In [19]:
# Dash Layout
app.layout = html.Div([
    html.H1("Stock Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[
        dcc.Tab(label='NSE-TATAGLOBAL Stock Data', children=[
            html.Div([
                html.H2("Actual closing price", style={"textAlign": "center"}),
                dcc.Graph(
                    id="Actual Data",
                    figure={
                        "data": [
                            go.Scatter(
                                x=valid.index,
                                y=valid["Close"],
                                mode='markers'
                            )
                        ],
                        "layout": go.Layout(
                            title='Scatter Plot',
                            xaxis={'title': 'Date'},
                            yaxis={'title': 'Closing Rate'}
                        )
                    }
                ),
                html.H2("LSTM Predicted closing price", style={"textAlign": "center"}),
                dcc.Graph(
                    id="Predicted Data",
                    figure={
                        "data": [
                            go.Scatter(
                                x=valid.index,
                                y=valid["Predictions"],
                                mode='markers'
                            )
                        ],
                        "layout": go.Layout(
                            title='Scatter Plot',
                            xaxis={'title': 'Date'},
                            yaxis={'title': 'Closing Rate'}
                        )
                    }
                )
            ])
        ]),

        dcc.Tab(label='Facebook Stock Data', children=[
            html.Div([
                html.H1("Facebook Stocks High vs Lows", style={'textAlign': 'center'}),
                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple', 'value': 'AAPL'},
                                      {'label': 'Facebook', 'value': 'FB'},
                                      {'label': 'Microsoft', 'value': 'MSFT'}],
                             multi=True, value=['FB'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='highlow'),

                html.H1("Facebook Market Volume", style={'textAlign': 'center'}),
                dcc.Dropdown(id='my-dropdown2',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple', 'value': 'AAPL'},
                                      {'label': 'Facebook', 'value': 'FB'},
                                      {'label': 'Microsoft', 'value': 'MSFT'}],
                             multi=True, value=['FB'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='volume')
            ], className="container"),
        ])
    ])
])

In [20]:
# Callback for the high/low stock graph
@app.callback(Output('highlow', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}
    trace1, trace2 = [], []

    for stock in selected_dropdown:
        trace1.append(go.Scatter(
            x=df[df["Stock"] == stock]["Date"],
            y=df[df["Stock"] == stock]["High"],
            mode='lines', opacity=0.7,
            name=f'High {dropdown[stock]}', textposition='bottom center'))

        trace2.append(go.Scatter(
            x=df[df["Stock"] == stock]["Date"],
            y=df[df["Stock"] == stock]["Low"],
            mode='lines', opacity=0.6,
            name=f'Low {dropdown[stock]}', textposition='bottom center'))

    data = [val for sublist in [trace1, trace2] for val in sublist]
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f"High and Low Prices for {', '.join(dropdown[i] for i in selected_dropdown)}",
            xaxis={"title": "Date", 'rangeslider': {'visible': True}, 'type': 'date'},
            yaxis={"title": "Price (USD)"}
        )
    }
    return figure

# Callback for the volume graph
@app.callback(Output('volume', 'figure'), [Input('my-dropdown2', 'value')])
def update_volume(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}
    trace1 = []

    for stock in selected_dropdown_value:
        trace1.append(go.Scatter(
            x=df[df["Stock"] == stock]["Date"],
            y=df[df["Stock"] == stock]["Volume"],
            mode='lines', opacity=0.7,
            name=f'Volume {dropdown[stock]}'
        ))

    figure = {
        'data': trace1,
        'layout': go.Layout(
            title=f"Market Volume for {', '.join(dropdown[i] for i in selected_dropdown_value)}",
            xaxis={"title": "Date", 'rangeslider': {'visible': True}, 'type': 'date'},
            yaxis={"title": "Transactions Volume"}
        )
    }
    return figure


In [21]:
if __name__ == '__main__':
    app.run_server(debug=True, dev_tools_hot_reload=False)
